In [1]:
import yt
from yt.units import Mpc
from yt.visualization.api import Streamlines
from yt.visualization.volume_rendering.api import LineSource
import numpy as np 

# Load the dataset
ds = yt.load("IsolatedGalaxy/galaxy0030/galaxy0030")

# Define c: the center of the box, N: the number of streamlines,
# scale: the spatial scale of the streamlines relative to the boxsize,
# and then pos: the random positions of the streamlines.
c = ds.domain_center
N = 10
scale = ds.domain_width[0]
pos_dx = np.random.random((N, 3)) * scale - scale / 2.0
pos = c + pos_dx

# Create streamlines of the 3D vector velocity and integrate them through
# the box defined above
streamlines = Streamlines(
    ds,
    pos,
    ("gas", "velocity_x"),
    ("gas", "velocity_y"),
    ("gas", "velocity_z"),
    length=1.0 * Mpc,
    get_magnitude=True,
)
streamlines.integrate_through_volume()

###############################################################################
# Coercing streamlines for creating LineSource objects
#
# LineSource expects expects line segments... this is not memory efficient, but 
# we'll explode each streamline into a series of line segments. The
# following function takes the positions of a single streamline and resamples
# and reshapes to turn an array of line segments.

def segment_single_streamline(pos_i):
    index_range = np.arange(0, pos_i.shape[0])
    line_indices = np.column_stack([index_range, index_range]).ravel()[1:-1]

    line_segments = pos_i[line_indices, :]
    n_line_segments = int(line_segments.size/6)
    return line_segments.reshape((n_line_segments, 2, 3))

# for example
pos = streamlines.streamlines
line_segments = segment_single_streamline(pos[0])
print(line_segments.shape)

# note the starting point of second position is the end point of first position
print(line_segments[0])
print(line_segments[1])

sc = yt.create_scene(ds)

# for each streamline, expand into line segments and
# add a LineSource
for sid in range(streamlines.streamlines.shape[0]):        
    line_segments = segment_single_streamline(pos[sid])    
    colors = np.ones([line_segments.shape[0], 4])
    # drop opacity -- need to fiddle with this depending on 
    # volume rendering 
    colors[:, -1] = 0.01
    lines = LineSource(line_segments, colors)    
    sc.add_source(lines)

sc.save(sigma_clip=4.0)


yt : [INFO     ] 2023-11-06 11:28:14,900 Parameters: current_time              = 0.0060000200028298
yt : [INFO     ] 2023-11-06 11:28:14,900 Parameters: domain_dimensions         = [32 32 32]
yt : [INFO     ] 2023-11-06 11:28:14,901 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2023-11-06 11:28:14,901 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2023-11-06 11:28:14,902 Parameters: cosmological_simulation   = 0
Parsing Hierarchy : 100%|██████████████████| 173/173 [00:00<00:00, 23194.43it/s]
yt : [INFO     ] 2023-11-06 11:28:14,922 Gathering a field list (this may take a moment.)
yt : [INFO     ] 2023-11-06 11:29:12,428 AMRKDTree rebuilt, Final Volume: 1.000000e+00


Streamlining: 100%|█████████████████████████████| 10/10 [00:02<00:00,  4.91it/s]


yt : [INFO     ] 2023-11-06 11:32:33,329 Setting default field to ('gas', 'density')
yt : [WARNING  ] 2023-11-06 11:32:33,345 No previously rendered image found, rendering now.
yt : [INFO     ] 2023-11-06 11:32:33,346 Rendering scene (Can take a while).
yt : [INFO     ] 2023-11-06 11:32:33,380 Creating volume


(8191, 2, 3)
[[0.68724773 0.18647061 0.78763723]
 [0.6872004  0.18655274 0.78756033]] code_length
[[0.6872004  0.18655274 0.78756033]
 [0.68715308 0.18663487 0.78748344]] code_length


yt : [INFO     ] 2023-11-06 11:32:35,129 Creating transfer function
yt : [INFO     ] 2023-11-06 11:32:35,129 Calculating data bounds. This may take a while. Set the TransferFunctionHelper.bounds to avoid this.
yt : [INFO     ] 2023-11-06 11:32:36,120 Saving rendered image to galaxy0030_Render_density.png
